In [1]:
import cv2
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from options.test_options import TestOptions
import os
from data import create_dataset
from models import create_model
from util.visualizer import save_images
from util import html,util

In [2]:
NAME = 'gen20'

# defaults = {
#     'dataroot': '../../hdr_out/06_Apr_14_16_50',
#     'model': 'hdr',
#     'dataset_mode': 'hdr',
#     'dataset_root': '../../hdr_out/06_Apr_14_16_50',
#     'embedding_save_dir': '',
#     'name': NAME
# }

defaults = {
    'dataroot': '../../generator_data/HFlickr/',
    'model': 'my',
    'dataset_mode': 'my',
    'dataset_root': '../../generator_data/HFlickr/',
    'embedding_save_dir': './checkpoints/emb2',
    'name': NAME
}

# defaults = {
#     'dataroot': '../../embedding_data/vidit/',
#     'model': 'emb',
#     'dataset_mode': 'emb',
#     'dataset_root': '../../embedding_data/vidit/',
# #     'embedding_save_dir': './checkpoints/hdr1/',
#     'name': NAME
# }



    
opt = TestOptions(defaults=defaults).parse()

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ../../generator_data/HFlickr/ 
             dataset_mode: my                            
             dataset_root: ../../generator_data/HFlickr/ 
                direction: AtoB                          
             display_port: 9333                          
          display_winsize: 256                           
       embedding_save_dir: ./checkpoints/emb2            
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 5  

In [3]:
# hard-code some parameters for test
opt.num_threads = 0   # test code only supports num_threads = 1
opt.batch_size = 1    # test code only supports batch_size = 1
opt.serial_batches = True  # disable data shuffling; comment this line if results on randomly chosen images are needed.
opt.no_flip = True    # no flip; comment this line if results on flipped images are needed.
opt.display_id = -1   # no visdom display; the test code saves the results to a HTML file.

In [4]:
assert opt.isTrain == False

In [5]:
dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
dataset_size = len(dataset)    # get the number of images in the dataset.
print('The number of training images = %d' % dataset_size)

model = create_model(opt)      # create a model given opt.model and other options
model.setup(opt)               # regular setup: load and print networks; create schedulers
total_iters = 0                # the total number of training iterations

loading test file
dataset [MyDataset] was created
The number of training images = 828
initialize network with normal
initialize network with normal
loading the model from ./checkpoints/emb2/latest_net_G.pth
---------- Networks initialized -------------
[Network G] Total number of parameters : 54.417 M
-----------------------------------------------
model [MyModel] was created
loading the model from ./checkpoints/gen20/latest_net_G.pth
---------- Networks initialized -------------
[Network G] Total number of parameters : 54.416 M
-----------------------------------------------


In [6]:
# create a website
web_dir = os.path.join(opt.results_dir, opt.name, '%s_%s' % (opt.phase, opt.epoch))  # define the website directory
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt.name, opt.phase, opt.epoch))

In [7]:
def to_img(tensor):
    np_img = np.squeeze(tensor.detach().to('cpu').numpy())
    np_img = np_img.transpose((1, 2, 0))
    np_img = cv2.cvtColor(np_img, cv2.COLOR_RGBA2RGB)
    return np_img
    

def display(visuals, epoch):
    real = to_img(visuals['real'])
    comp = to_img(visuals['comp'])
    harmonized = to_img(visuals['harmonized'])
    
    print(f'Epoch {epoch} composition/real/harmonized')
    display = np.concatenate([comp, real, harmonized], axis=1)
    fig = plt.figure(figsize=(15, 15))
    plt.imshow(display)
    plt.show()

In [8]:
test_loss = 0.0

if opt.eval:
    model.eval()

for i, data in enumerate(dataset):
#     if i >= opt.num_test:
#         break
    model.set_input(data)  # unpack data from data loader
    model.test()           # run inference
    visuals = model.get_current_visuals()  # get image results
    for label, im_data in visuals.items():
        if label=='harmonized':
            #print(label)
            #print(im_data)
            im = util.tensor2im(im_data)
            img_path = str(data['img_path'])
            raw_name = img_path.split('/')[-1]
            raw_name = raw_name.replace(('[\''),'')
            raw_name = raw_name.replace(('.jpg\']'),'.jpg')
            image_name = '%s' % raw_name
            save_path = os.path.join(opt.results_dir+opt.name+'/test_latest/images/', image_name)
            util.save_image(im, save_path, aspect_ratio=opt.aspect_ratio)
            print('processing (%04d)-th image... %s' % (i, img_path))
            break
    losses = model.get_current_losses()
    test_loss += losses['G_L1']
    img_path = model.get_image_paths()     # get image paths # Added by Mia
    if i % 10 == 0:  # save images to an HTML file
        print('saving (%04d)-th image... %s' % (i, img_path))
        save_images(webpage, visuals, img_path, aspect_ratio=opt.aspect_ratio, width=opt.display_winsize)
webpage.save()  # save the HTML
print(test_loss)

processing (0000)-th image... ['../../generator_data/HFlickr/composite_images/f4005_1_1.jpg']
saving (0000)-th image... ['../../generator_data/HFlickr/composite_images/f4005_1_1.jpg']
processing (0001)-th image... ['../../generator_data/HFlickr/composite_images/f2508_1_1.jpg']
processing (0002)-th image... ['../../generator_data/HFlickr/composite_images/f2508_1_2.jpg']
processing (0003)-th image... ['../../generator_data/HFlickr/composite_images/f3885_1_2.jpg']
processing (0004)-th image... ['../../generator_data/HFlickr/composite_images/f3885_1_1.jpg']
processing (0005)-th image... ['../../generator_data/HFlickr/composite_images/f4493_1_2.jpg']
processing (0006)-th image... ['../../generator_data/HFlickr/composite_images/f4493_1_1.jpg']
processing (0007)-th image... ['../../generator_data/HFlickr/composite_images/f4432_1_1.jpg']
processing (0008)-th image... ['../../generator_data/HFlickr/composite_images/f4432_1_2.jpg']
processing (0009)-th image... ['../../generator_data/HFlickr/com

processing (0081)-th image... ['../../generator_data/HFlickr/composite_images/f83_1_1.jpg']
processing (0082)-th image... ['../../generator_data/HFlickr/composite_images/f1015_1_1.jpg']
processing (0083)-th image... ['../../generator_data/HFlickr/composite_images/f1015_1_2.jpg']
processing (0084)-th image... ['../../generator_data/HFlickr/composite_images/f4252_1_1.jpg']
processing (0085)-th image... ['../../generator_data/HFlickr/composite_images/f4252_1_2.jpg']
processing (0086)-th image... ['../../generator_data/HFlickr/composite_images/f5153_1_1.jpg']
processing (0087)-th image... ['../../generator_data/HFlickr/composite_images/f5153_1_2.jpg']
processing (0088)-th image... ['../../generator_data/HFlickr/composite_images/f1219_1_2.jpg']
processing (0089)-th image... ['../../generator_data/HFlickr/composite_images/f1219_1_1.jpg']
processing (0090)-th image... ['../../generator_data/HFlickr/composite_images/f3720_1_1.jpg']
saving (0090)-th image... ['../../generator_data/HFlickr/compo

processing (0161)-th image... ['../../generator_data/HFlickr/composite_images/f1534_1_1.jpg']
processing (0162)-th image... ['../../generator_data/HFlickr/composite_images/f1534_1_2.jpg']
processing (0163)-th image... ['../../generator_data/HFlickr/composite_images/f4222_1_1.jpg']
processing (0164)-th image... ['../../generator_data/HFlickr/composite_images/f4222_1_2.jpg']
processing (0165)-th image... ['../../generator_data/HFlickr/composite_images/f4007_1_1.jpg']
processing (0166)-th image... ['../../generator_data/HFlickr/composite_images/f4007_1_2.jpg']
processing (0167)-th image... ['../../generator_data/HFlickr/composite_images/f144_1_1.jpg']
processing (0168)-th image... ['../../generator_data/HFlickr/composite_images/f144_1_2.jpg']
processing (0169)-th image... ['../../generator_data/HFlickr/composite_images/f3267_1_1.jpg']
processing (0170)-th image... ['../../generator_data/HFlickr/composite_images/f5231_2_2.jpg']
saving (0170)-th image... ['../../generator_data/HFlickr/compo

processing (0241)-th image... ['../../generator_data/HFlickr/composite_images/f4971_1_2.jpg']
processing (0242)-th image... ['../../generator_data/HFlickr/composite_images/f462_1_2.jpg']
processing (0243)-th image... ['../../generator_data/HFlickr/composite_images/f462_1_1.jpg']
processing (0244)-th image... ['../../generator_data/HFlickr/composite_images/f5129_1_1.jpg']
processing (0245)-th image... ['../../generator_data/HFlickr/composite_images/f5129_1_2.jpg']
processing (0246)-th image... ['../../generator_data/HFlickr/composite_images/f1106_1_2.jpg']
processing (0247)-th image... ['../../generator_data/HFlickr/composite_images/f1106_1_1.jpg']
processing (0248)-th image... ['../../generator_data/HFlickr/composite_images/f1958_1_2.jpg']
processing (0249)-th image... ['../../generator_data/HFlickr/composite_images/f1958_1_1.jpg']
processing (0250)-th image... ['../../generator_data/HFlickr/composite_images/f1961_1_1.jpg']
saving (0250)-th image... ['../../generator_data/HFlickr/compo

processing (0321)-th image... ['../../generator_data/HFlickr/composite_images/f3555_1_2.jpg']
processing (0322)-th image... ['../../generator_data/HFlickr/composite_images/f5233_2_1.jpg']
processing (0323)-th image... ['../../generator_data/HFlickr/composite_images/f5233_2_2.jpg']
processing (0324)-th image... ['../../generator_data/HFlickr/composite_images/f18_1_1.jpg']
processing (0325)-th image... ['../../generator_data/HFlickr/composite_images/f18_1_2.jpg']
processing (0326)-th image... ['../../generator_data/HFlickr/composite_images/f1608_1_2.jpg']
processing (0327)-th image... ['../../generator_data/HFlickr/composite_images/f1608_1_1.jpg']
processing (0328)-th image... ['../../generator_data/HFlickr/composite_images/f310_1_2.jpg']
processing (0329)-th image... ['../../generator_data/HFlickr/composite_images/f310_1_1.jpg']
processing (0330)-th image... ['../../generator_data/HFlickr/composite_images/f1774_1_1.jpg']
saving (0330)-th image... ['../../generator_data/HFlickr/composite

...

processing (0481)-th image... ['../../generator_data/HFlickr/composite_images/f1795_1_2.jpg']
processing (0482)-th image... ['../../generator_data/HFlickr/composite_images/f1795_1_1.jpg']
processing (0483)-th image... ['../../generator_data/HFlickr/composite_images/f227_1_1.jpg']
processing (0484)-th image... ['../../generator_data/HFlickr/composite_images/f227_1_2.jpg']
processing (0485)-th image... ['../../generator_data/HFlickr/composite_images/f323_1_2.jpg']
processing (0486)-th image... ['../../generator_data/HFlickr/composite_images/f323_1_1.jpg']
processing (0487)-th image... ['../../generator_data/HFlickr/composite_images/f1172_1_1.jpg']
processing (0488)-th image... ['../../generator_data/HFlickr/composite_images/f1172_1_2.jpg']
processing (0489)-th image... ['../../generator_data/HFlickr/composite_images/f24_1_2.jpg']
processing (0490)-th image... ['../../generator_data/HFlickr/composite_images/f24_1_1.jpg']
saving (0490)-th image... ['../../generator_data/HFlickr/composite_i

processing (0561)-th image... ['../../generator_data/HFlickr/composite_images/f406_1_1.jpg']
processing (0562)-th image... ['../../generator_data/HFlickr/composite_images/f406_1_2.jpg']
processing (0563)-th image... ['../../generator_data/HFlickr/composite_images/f5212_1_1.jpg']
processing (0564)-th image... ['../../generator_data/HFlickr/composite_images/f5212_1_2.jpg']
processing (0565)-th image... ['../../generator_data/HFlickr/composite_images/f1959_1_2.jpg']
processing (0566)-th image... ['../../generator_data/HFlickr/composite_images/f1959_1_1.jpg']
processing (0567)-th image... ['../../generator_data/HFlickr/composite_images/f1439_1_2.jpg']
processing (0568)-th image... ['../../generator_data/HFlickr/composite_images/f1439_1_1.jpg']
processing (0569)-th image... ['../../generator_data/HFlickr/composite_images/f3300_1_1.jpg']
processing (0570)-th image... ['../../generator_data/HFlickr/composite_images/f3300_1_2.jpg']
saving (0570)-th image... ['../../generator_data/HFlickr/compo

processing (0641)-th image... ['../../generator_data/HFlickr/composite_images/f2811_1_1.jpg']
processing (0642)-th image... ['../../generator_data/HFlickr/composite_images/f2811_1_2.jpg']
processing (0643)-th image... ['../../generator_data/HFlickr/composite_images/f2974_1_2.jpg']
processing (0644)-th image... ['../../generator_data/HFlickr/composite_images/f2974_1_1.jpg']
processing (0645)-th image... ['../../generator_data/HFlickr/composite_images/f3260_1_2.jpg']
processing (0646)-th image... ['../../generator_data/HFlickr/composite_images/f3260_1_1.jpg']
processing (0647)-th image... ['../../generator_data/HFlickr/composite_images/f1037_1_1.jpg']
processing (0648)-th image... ['../../generator_data/HFlickr/composite_images/f1037_1_2.jpg']
processing (0649)-th image... ['../../generator_data/HFlickr/composite_images/f1570_1_2.jpg']
processing (0650)-th image... ['../../generator_data/HFlickr/composite_images/f1570_1_1.jpg']
saving (0650)-th image... ['../../generator_data/HFlickr/com

processing (0721)-th image... ['../../generator_data/HFlickr/composite_images/f3794_1_1.jpg']
processing (0722)-th image... ['../../generator_data/HFlickr/composite_images/f4297_1_1.jpg']
processing (0723)-th image... ['../../generator_data/HFlickr/composite_images/f4297_1_2.jpg']
processing (0724)-th image... ['../../generator_data/HFlickr/composite_images/f692_1_1.jpg']
processing (0725)-th image... ['../../generator_data/HFlickr/composite_images/f692_1_2.jpg']
processing (0726)-th image... ['../../generator_data/HFlickr/composite_images/f2832_1_1.jpg']
processing (0727)-th image... ['../../generator_data/HFlickr/composite_images/f2832_1_2.jpg']
processing (0728)-th image... ['../../generator_data/HFlickr/composite_images/f2021_1_1.jpg']
processing (0729)-th image... ['../../generator_data/HFlickr/composite_images/f2021_1_2.jpg']
processing (0730)-th image... ['../../generator_data/HFlickr/composite_images/f129_1_2.jpg']
saving (0730)-th image... ['../../generator_data/HFlickr/compos

processing (0801)-th image... ['../../generator_data/HFlickr/composite_images/f2800_1_1.jpg']
processing (0802)-th image... ['../../generator_data/HFlickr/composite_images/f2800_1_2.jpg']
processing (0803)-th image... ['../../generator_data/HFlickr/composite_images/f3638_1_2.jpg']
processing (0804)-th image... ['../../generator_data/HFlickr/composite_images/f1070_1_1.jpg']
processing (0805)-th image... ['../../generator_data/HFlickr/composite_images/f1070_1_2.jpg']
processing (0806)-th image... ['../../generator_data/HFlickr/composite_images/f4169_1_1.jpg']
processing (0807)-th image... ['../../generator_data/HFlickr/composite_images/f4169_1_2.jpg']
processing (0808)-th image... ['../../generator_data/HFlickr/composite_images/f1267_1_1.jpg']
processing (0809)-th image... ['../../generator_data/HFlickr/composite_images/f1267_1_2.jpg']
processing (0810)-th image... ['../../generator_data/HFlickr/composite_images/f466_1_2.jpg']
saving (0810)-th image... ['../../generator_data/HFlickr/comp